In [ ]:
from skimage import segmentation, color
from skimage import morphology
import matplotlib.pyplot as plt
import matplotlib.colors
import math
import numpy as np
import os
import pandas as pd


In [40]:
# Just the structure of the program, these's a function defined for the same reason!!!
# rgb_img = plt.imread(img_path)[:,:,:3]
# mask = plt.imread(mask_path)
    
# #Average colour on the lesion:
# img_avg_lesion = rgb_img.copy()
# for i in range(3):
#     channel = img_avg_lesion[:,:,i]
#     mean = np.mean(channel[mask.astype(bool)])
#     channel[mask.astype(bool)] = mean
#     img_avg_lesion[:,:,i] = channel

# # cropping the image according to the mask
# lesion_coords = np.where(mask != 0)
# min_x = min(lesion_coords[0])
# max_x = max(lesion_coords[0])
# min_y = min(lesion_coords[1])
# max_y = max(lesion_coords[1])
# cropped_lesion = rgb_img[min_x:max_x,min_y:max_y]

# # Segment the cropped lesion with SLIC (without boundaries)
# img_avg_lesion_1 = rgb_img.copy()
# segments_1 = segmentation.slic(cropped_lesion, compactness = 50, n_segments=10, sigma=3,
#                             start_label=1)
# out_1 = color.label2rgb(segments_1, cropped_lesion, kind='avg')
# img_avg_lesion_1[min_x:max_x,min_y:max_y] = out_1
# img_avg_lesion[mask == 0] = rgb_img[mask==0]

# # Segment the cropped lesion with SLIC (with boundaries)
# img_avg_lesion_2 = rgb_img.copy()
# segments_2 = segmentation.slic(cropped_lesion, compactness= 50, n_segments=10, sigma=3,
#                             start_label=1)
# out_2 = color.label2rgb(segments_2, cropped_lesion, kind='avg')
# out1 = segmentation.mark_boundaries(out_2, segments_2)
# img_avg_lesion_2[min_x:max_x,min_y:max_y] = out1
# img_avg_lesion_2[mask == 0] = rgb_img[mask==0]


In [41]:
# #Visualization between img_avg_lesion_1 and 2:
# # print(f'This is result 2:{img_avg_lesion_2}')
# plt.title(f"SLIC 2")
# plt.imshow(img_avg_lesion_2)
# # print(f'This is result 1:{img_avg_lesion_1}')
# # plt.title(f"SLIC with 1  ")
# # plt.imshow(img_avg_lesion_1)


In [42]:
# # calculate the standard deviation in the HSV channel, already inclued in the function process...
# img_avg_lesion_1_hsv = matplotlib.colors.rgb_to_hsv(img_avg_lesion_1)
# img_avg_lesion_2_hsv = matplotlib.colors.rgb_to_hsv(img_avg_lesion_2)
# mask_hsv = mask.copy()

In [43]:
def calculate_hsv_deviations(hsv_image, mask):
    """Calculate the standard deviation for each HSV channel within the masked area."""
    mask_bool = mask > 0 #mask_bool, has True values where the mask is greater than 0 (indicating the area of interest) and False elsewhere.
    std_devs = []
    deviations = []
    std_devs = {'Hue': 0, 'Saturation': 0, 'Value': 0, 'Uniformity_Score': 0}
    for i, channel_name in enumerate(['Hue', 'Saturation', 'Value']):  # Iterate over HUE, SATURATION, VALUE channels
        channel = hsv_image[:, :, i] # [x,y, H/S/V] where x and y coordinates
        masked_channel = channel[mask_bool] # Apply mask
        std_dev = np.std(masked_channel)
        deviations.append(std_dev)
        std_devs[channel_name] = std_dev
    std_devs['Uniformity_Score'] = np.mean(deviations)
    return std_devs

In [44]:
#Testing the results

# x = calculate_hsv_deviations(img_avg_lesion_1_hsv,mask_hsv)
# z = calculate_hsv_deviations(img_avg_lesion_2_hsv,mask_hsv)
# print(x)
# z

In [52]:
#for loop for processing all the pictures and save it to csv:
def process_images_with_hsv_uniformity(img_path, mask_path_o):
    '''
    This function is processing the images in HSV chanel to get 
    the standard deviation on the uniformity, gives a DF as result
    '''
    HSV_score = {'img_id':[],
                 'hue':[],
                 'saturation':[],
                 'value':[],
                'hsv_uniformity':[]}
    for filename in os.listdir(img_path):
        if filename.endswith(".png"):
            base_name = filename[:-4]
            image_path = os.path.join(img_path, filename)
            mask_name = base_name + '_mask.png'
            mask_path = os.path.join(mask_path_o, mask_name)
            
            # original_image = cv2.imread(image_path)
            rgb_img = plt.imread(image_path)[:,:,:3]
            mask = plt.imread(mask_path)
            # mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            #rgb to hsv
            
            if filename is None or mask_name is None:
                HSV_score['img_id'].append(filename)
                HSV_score['hue','saturation','value','hsv_uniformity'].append('N/A')
                continue
                
            #Average colour on the lesion:
            img_avg_lesion = rgb_img.copy()
            for i in range(3):
                channel = img_avg_lesion[:,:,i]
                mean = np.mean(channel[mask.astype(bool)])
                channel[mask.astype(bool)] = mean
                img_avg_lesion[:,:,i] = channel

            # cropping the image according to the mask
            lesion_coords = np.where(mask != 0)
            min_x = min(lesion_coords[0])
            max_x = max(lesion_coords[0])
            min_y = min(lesion_coords[1])
            max_y = max(lesion_coords[1])
            cropped_lesion = rgb_img[min_x:max_x,min_y:max_y]

            # Segment the cropped lesion with SLIC (without boundaries)
            img_avg_lesion_1 = rgb_img.copy()
            segments_1 = segmentation.slic(cropped_lesion, compactness = 50, n_segments=10, sigma=3,
                                        start_label=1)
            out_1 = color.label2rgb(segments_1, cropped_lesion, kind='avg')
            img_avg_lesion_1[min_x:max_x,min_y:max_y] = out_1
            img_avg_lesion[mask == 0] = rgb_img[mask==0]            

            img_avg_lesion_1_hsv = matplotlib.colors.rgb_to_hsv(img_avg_lesion_1)
            
            uniformity = calculate_hsv_deviations(img_avg_lesion_1_hsv, mask)
            HSV_score['img_id'].append(filename)
            HSV_score['hue'].append(uniformity['Hue'])
            HSV_score['saturation'].append(uniformity['Saturation'])
            HSV_score['value'].append(uniformity['Value'])
            HSV_score['hsv_uniformity'].append(uniformity['Uniformity_Score'])
    HSV_score_df = pd.DataFrame(HSV_score)
    return  HSV_score_df

In [54]:
img_path_test = 'C:\\Users\\Ivett\\OneDrive\\Documents\\ITU_DS\\GIT\\projects-in-data-science\\data\\images'
mask_path_test = 'C:\\Users\\Ivett\\OneDrive\\Documents\\ITU_DS\\GIT\\projects-in-data-science\\data\\masks'
data = process_images_with_hsv_uniformity(img_path_test,mask_path_test)

In [55]:
meta_data_csv = 'C:\\Users\\Ivett\\OneDrive\\Documents\\ITU_DS\\GIT\\projects-in-data-science\\data\\metadata.csv'

def creating_basic_df(meta_data_csv):
    '''
    This function is preparing the main df for the features csv
    '''
    meta_data = pd.read_csv(meta_data_csv)
    meta_data = meta_data[['img_id','diagnostic']]
    meta_data['cancer'] = 0
    meta_data['cancer'] = meta_data['cancer'].fillna(0)
    meta_data.loc[meta_data['diagnostic'].isin(['BCC', 'SCC', 'MEL']), 'cancer'] = 1
   
    return meta_data

meta_df = creating_basic_df(meta_data_csv)
combined_df = pd.merge(meta_df,data, on='img_id')
combined_df.to_csv("test.csv", index = False)
